# Building a Data Pipeline in Databricks and using Spark
---

#### Senior Data Scientist.: Dr. Eddy Giusepe Chirinos Isidro

# 🚀 Configuração do Databricks Connect

Este notebook está configurado para usar **Databricks Connect**, permitindo executar código localmente mas processando dados no cluster Databricks.

## 🔐 Informações necessárias:

1. **Server hostname**: URL do seu workspace Databricks
2. **Personal Access Token**: Token de acesso (User Settings → Developer → Access Tokens)
3. **Cluster ID**: ID do cluster ativo (copie da URL quando abrir um cluster)

## 📋 Como configurar:

### 1️⃣ Criar variáveis de ambiente (recomendado):
```bash
export DATABRICKS_HOST="https://seu-workspace.cloud.databricks.com"
export DATABRICKS_TOKEN="dapi-seu-token-aqui"  
export DATABRICKS_CLUSTER_ID="cluster-id-aqui"
```

### 2️⃣ Ou configure diretamente no código abaixo ⬇️


# 📋 COMO OBTER AS CREDENCIAIS DO DATABRICKS

## 🔍 1. Server Hostname
- Va para seu workspace Databricks
- Copie a URL do navegador (ex: `https://dbc-a1b2c3d4-e5f6.cloud.databricks.com`)

## 🔑 2. Personal Access Token
1. No Databricks workspace → clique no seu **avatar** (canto superior direito)
2. **User Settings**
3. **Developer** (no menu lateral)
4. **Access Tokens**
5. **Generate New Token**
6. Dê um nome (ex: "Local Development")
7. Defina expiração (recomendo 90 dias)
8. **Generate** → copie o token (começa com `dapi-...`)

## 💻 3. Cluster ID
1. No Databricks workspace → **Compute** (menu lateral)
2. Clique no cluster que deseja usar
3. Copie o **Cluster ID** da URL ou das configurações do cluster

## 🔒 4. Configurar de forma segura
### Opção A: Variáveis de ambiente (recomendado)
```bash
export DATABRICKS_HOST="https://seu-workspace.cloud.databricks.com"
export DATABRICKS_TOKEN="dapi-seu-token-aqui"
export DATABRICKS_CLUSTER_ID="cluster-id-aqui"
```

### Opção B: Arquivo .env (alternativa)
Crie um arquivo `.env` na pasta do projeto:
```
DATABRICKS_HOST=https://seu-workspace.cloud.databricks.com
DATABRICKS_TOKEN=dapi-seu-token-aqui
DATABRICKS_CLUSTER_ID=cluster-id-aqui
```

**⚠️ IMPORTANTE**: Nunca faça commit de tokens/credenciais para Git!

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

# Carregar credenciais
_ = load_dotenv(find_dotenv())
databricks_host = os.environ['DATABRICKS_HOST']
databricks_token = os.environ['DATABRICKS_TOKEN']
databricks_cluster_id = os.environ['DATABRICKS_CLUSTER_ID']

print("🔗 Conectando ao Databricks...")
print(f"📍 Host: {databricks_host}")
print(f"🆔 Cluster: {databricks_cluster_id}")

# Conectar ao Databricks
from databricks.connect import DatabricksSession

spark = DatabricksSession.builder \
    .remote(
        host=databricks_host,
        token=databricks_token,
        cluster_id=databricks_cluster_id
    ) \
    .getOrCreate()

print("✅ Conectado ao Databricks com sucesso!")
print(f"🔧 Versão Spark: {spark.version}")

# 📁 CARREGAR DADOS DO VOLUME DATABRICKS
volume_path = "/Volumes/workspace/default_eddy/volumeeddy-tmp-sampledata/sample_data.csv"
print(f"\n📂 Carregando dados do Volume: {volume_path}")

df_spark = spark.read.csv(volume_path, header=True, inferSchema=True)

print("🎉 DADOS DO VOLUME CARREGADOS COM SUCESSO!")
print("\n📊 Primeiros registros:")
df_spark.show()

print(f"\n📈 Informações do Dataset:")
print(f"   • Linhas: {df_spark.count()}")
print(f"   • Colunas: {len(df_spark.columns)}")
print(f"   • Nomes das Colunas: {df_spark.columns}")

print("\n📋 Schema do Dataset:")
df_spark.printSchema()

# Converter para Pandas para análises adicionais (opcional)
df_pandas = df_spark.toPandas()
print(f"\n✅ Dados também disponíveis como Pandas DataFrame para análises locais!")


🔗 Conectando ao Databricks...
📍 Host: https://dbc-15e1a878-daba.cloud.databricks.com
🆔 Cluster: 0919-175003-lz63w6ap-v2n
✅ Conectado ao Databricks com sucesso!
🔧 Versão Spark: 4.0.0

📂 Carregando dados do Volume: /Volumes/workspace/default_eddy/volumeeddy-tmp-sampledata/sample_data.csv
🎉 DADOS DO VOLUME CARREGADOS COM SUCESSO!

📊 Primeiros registros:
+-------+---+-----------+
|   Name|Age|       City|
+-------+---+-----------+
|  Alice| 25|   New York|
|    Bob| 17|Los Angeles|
|Charlie| 35|    Chicago|
|  Diana| 16|    Houston|
| Edward| 45|    Phoenix|
+-------+---+-----------+


📈 Informações do Dataset:
   • Linhas: 5
   • Colunas: 3
   • Nomes das Colunas: ['Name', 'Age', 'City']

📋 Schema do Dataset:
root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- City: string (nullable = true)


✅ Dados também disponíveis como Pandas DataFrame para análises locais!


In [2]:
# 📊 ANÁLISE AVANÇADA DOS DADOS DO VOLUME DATABRICKS
print("="*70)
print("📊 ANÁLISE COMPLETA DOS DADOS DO VOLUME DATABRICKS")
print("="*70)

# Estatísticas descritivas com Spark
print("\n📊 Estatísticas Descritivas (processadas no cluster Databricks):")
df_spark.describe().show()

# Análises com Spark SQL
print("\n🔍 Análises com Spark SQL:")

# Registrar como view temporária para usar SQL
df_spark.createOrReplaceTempView("pessoas")

# 1. Análise de idade
print("\n👥 Análise de Idade:")
idade_stats = spark.sql("""
    SELECT 
        AVG(Age) as idade_media,
        MIN(Age) as idade_minima,
        MAX(Age) as idade_maxima,
        COUNT(*) as total_pessoas
    FROM pessoas
""")
idade_stats.show()

# 2. Maiores de idade
print("\n🔞 Pessoas maiores de idade:")
adults = spark.sql("SELECT * FROM pessoas WHERE Age >= 18")
adults.show()

adult_count = adults.count()
total_count = df_spark.count()
print(f"📈 {adult_count} de {total_count} pessoas são maiores de idade ({adult_count/total_count*100:.1f}%)")

# 3. Análise por cidade
print("\n🏙️ Contagem por Cidade:")
city_analysis = spark.sql("""
    SELECT City, COUNT(*) as quantidade
    FROM pessoas 
    GROUP BY City 
    ORDER BY quantidade DESC
""")
city_analysis.show()

# 4. Análises avançadas com Window Functions
print("\n📈 Análises Avançadas:")
advanced_analysis = spark.sql("""
    SELECT 
        Name,
        Age,
        City,
        CASE 
            WHEN Age >= 18 THEN 'Adulto'
            ELSE 'Menor de idade'
        END as categoria_idade,
        ROW_NUMBER() OVER (PARTITION BY City ORDER BY Age DESC) as ranking_idade_cidade
    FROM pessoas
    ORDER BY City, Age DESC
""")
advanced_analysis.show()

# Converter para Pandas para análises complementares
print("\n🐼 Análises complementares com Pandas:")
df_pandas = df_spark.toPandas()
print(f"   • Dataset convertido: {df_pandas.shape[0]} linhas, {df_pandas.shape[1]} colunas")
print(f"   • Idade média: {df_pandas['Age'].mean():.2f} anos")
print(f"   • Cidades únicas: {df_pandas['City'].nunique()}")

print("\n✅ Pipeline ETL com Databricks Volume concluído com sucesso!")
print("   • 🌩️ Processamento: Cluster Databricks")
print("   • 📁 Fonte: Volume Databricks")
print(f"   • 📊 Registros: {df_spark.count()}")
print(f"   • 🚀 Engine: Spark {spark.version}")
print("   • 💾 Disponível em: Spark DataFrame + Pandas DataFrame")


📊 ANÁLISE COMPLETA DOS DADOS DO VOLUME DATABRICKS

📊 Estatísticas Descritivas (processadas no cluster Databricks):
+-------+------+------------------+-------+
|summary|  Name|               Age|   City|
+-------+------+------------------+-------+
|  count|     5|                 5|      5|
|   mean|  NULL|              27.6|   NULL|
| stddev|  NULL|12.361229712289955|   NULL|
|    min| Alice|                16|Chicago|
|    max|Edward|                45|Phoenix|
+-------+------+------------------+-------+


🔍 Análises com Spark SQL:

👥 Análise de Idade:
+-----------+------------+------------+-------------+
|idade_media|idade_minima|idade_maxima|total_pessoas|
+-----------+------------+------------+-------------+
|       27.6|          16|          45|            5|
+-----------+------------+------------+-------------+


🔞 Pessoas maiores de idade:
+-------+---+--------+
|   Name|Age|    City|
+-------+---+--------+
|  Alice| 25|New York|
|Charlie| 35| Chicago|
| Edward| 45| Phoenix|
+-